In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes trl==0.8.6 faiss-cpu

In [ ]:
import huggingface_hub
import torch
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import gc
import re
import pandas as pd
huggingface_hub.login(token="token")

In [ ]:
Q_TKN = '<  question>' #깃허브에 업로드할 때 생략되어 나와서 tab 추가함
C_TKN = '<  context>'
A_TKN = '<  sys>'
EOS = '<|endoftext|>'
SEP = '<|sep|>'
def make_prompt(question, context):
    context = re.sub('[.]{2,5}', '', context)
    prompt = C_TKN + context.replace('.', f".{SEP} ")
    prompt = prompt.replace(f'{SEP} "', '"').replace(f"{SEP} '", "'")
    prompt += f"""{Q_TKN}{question}{SEP} {A_TKN}"""
    return prompt

# 데이터 불러오기

In [ ]:
import pandas as pd
df = pd.read_csv('pan_qna.csv')
df2 = pd.read_csv('gujo_qna.csv')
source = pd.concat([df, df2], axis=0).reset_index(drop=True)
source.to_csv('law_qna.csv', index=False)

In [ ]:
df = pd.read_csv('law_qna.csv')
df

,question,answer
0,개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에...,"행정심판법 제2조 제1항 및 제3조 제1항의 규정에 의하면, 행정심판은 행정청이 행..."
1,종전의 규정에 의해 허가를 받은 자에게 신 규정을 적용하여 매립목적변경의 대상이 아...,기타 주변여건의 변화 등으로 인하여 매립목적의 변경이 불가피한 경우인지 등을 종합적...
2,"군 복무중 질병이 발병 또는 악화되어 공상판정을 받고 전역하였고, 이에 대하여 ○○...",청구인이 군 복무중 이 건 질병들이 발병 또는 악화되어 육군에서 공상판정을 받고 전...
3,토지임차인의 지상물매수청구권을 배제하기로 하여 임차인에게 불리한 약정은 그 효력이 ...,임차인이 임대차계약을 체결할 당시 건물 기타 지상시설 일체를 포기하기로 하였다 해도...
4,낙뢰의 위험이 상당한 정도로 예상되는 경우 체육시설업자인 골프장 운영자에게 이용자에...,체육시설업자로서는 낙뢰의 위험이 상당한 정도로 예상되는 경우에는 이용자에 대하여 피...
...,...,...
102853,권한쟁의심판,국가기관 상호간이나 지방자치단체 상호간 또는 국가기관과 지방자치단체 사이에 권한이 ...
102854,국민 개인이 대법원장을 상대로 제기한 권한쟁의심판의 가능성,"헌법재판소는, 헌법재판소가 권한쟁의심판을 청구할 수 있는 국가기관의 종류와 범위에 ..."
102855,교육감과 해당 지방자치단체 사이의 내부적 분쟁과 관련한 권한쟁의심판청구의 적법 여부,"헌법재판소는, 헌법 제111조 제1항 제4호는 지방자치단체 상호간의 권한쟁의에 관한..."
102856,정당해산결정에 대하여 재심이 허용되는지 여부,"헌법재판소는, 정당해산심판은 원칙적으로 해당 정당에게만 그 효력이 미치며, 정당해산..."


In [ ]:
print(df['question'][0])
print(df['answer'][0])
print(make_prompt(df['question'][0], df['answer'][0]))

개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위로 보아야 하는가?
행정심판법 제2조 제1항 및 제3조 제1항의 규정에 의하면, 행정심판은 행정청이 행하는 구체적 사실에 관한 법집행으로서의 공권력의 행사와 그밖에 이에 준하는 행정작용을 대상으로 제기할 수 있도록 되어 있는바, 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위라고 할 것이므로 이 건 청구는 행정심판의 대상이 아닌 사항에 대하여 제기된 부적법한 심판청구라 할 것이다.
<context>행정심판법 제2조 제1항 및 제3조 제1항의 규정에 의하면, 행정심판은 행정청이 행하는 구체적 사실에 관한 법집행으로서의 공권력의 행사와 그밖에 이에 준하는 행정작용을 대상으로 제기할 수 있도록 되어 있는바, 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위라고 할 것이므로 이 건 청구는 행정심판의 대상이 아닌 사항에 대하여 제기된 부적법한 심판청구라 할 것이다.<|sep|> <question>개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위로 보아야 하는가?<|sep|> <sys>


# 추론

## 파이프라인 이용

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
model_id = 'won2133/polyglot-ko-1.3b-RAG-COT-ko'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = model = AutoModelForCausalLM.from_pretrained(model_id)
def make_inference_pipeline():
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

In [ ]:
prompt = make_prompt(df['question'][0], df['answer'][0])
print(prompt)
hf_pipe = make_inference_pipeline()
hf_pipe(prompt, do_sample=False,
    return_full_text=False, max_length=400, truncation=True)

Device set to use cuda:0


<context>행정심판법 제2조 제1항 및 제3조 제1항의 규정에 의하면, 행정심판은 행정청이 행하는 구체적 사실에 관한 법집행으로서의 공권력의 행사와 그밖에 이에 준하는 행정작용을 대상으로 제기할 수 있도록 되어 있는바, 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위라고 할 것이므로 이 건 청구는 행정심판의 대상이 아닌 사항에 대하여 제기된 부적법한 심판청구라 할 것이다.<|sep|> <question>개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위로 보아야 하는가?<|sep|> <sys>


[{'generated_text': '개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위로 보아야 하므로, 이 건 청구는 행정심판의 대상이 아닌 사항에 대하여 제기된 부적법한 심판청구로 보아야 한다. 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위가 사법상 계약에 관한 행위로 보아야 하는 이유는 무엇인가? 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위로 보아야 하는 이유는, 이는 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위가 사법상 계약에 관한 행위로 간주되기 때문이다.  따라서, 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위로 간주되어, 이는 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 행위가 사법상 계약에 관한 행위로 간주되는 것을 의미한다.  이러한 행위는'}]

## 파이프라인 직접 만들기

In [ ]:
import torch
from torch.nn.functional import softmax
from transformers import AutoModelForCausalLM, AutoTokenizer
class CustomPipeline:
    def __init__(self):
      model.eval()

    def __call__(self, q, context):
        s = make_prompt(q, context)
        print(s)
        tok_list = tokenizer.encode(s)
        result = []
        print(tok_list)
        with torch.no_grad():
          for i in range(100):
            input_ids = torch.LongTensor(tok_list).unsqueeze(dim=0)
            pred = model(input_ids)
            logits = pred.logits
            gen = torch.argmax(logits, dim=-1).squeeze().numpy().tolist()[-1]
            #print(tokenizer.decode(gen))
            tok_list.append(gen)
            result.append(gen)
            #종료 토큰이나 문장 구분 토큰이 나오면 루프 멈추기
            if gen == tokenizer.encode(EOS)[0] or gen == tokenizer.encode(SEP)[0]:
              break
        return tokenizer.decode(result, skip_special_tokens=True)

custom_pipeline = CustomPipeline()

In [ ]:
custom_pipeline(df['question'][0], df['answer'][0])

<context>행정심판법 제2조 제1항 및 제3조 제1항의 규정에 의하면, 행정심판은 행정청이 행하는 구체적 사실에 관한 법집행으로서의 공권력의 행사와 그밖에 이에 준하는 행정작용을 대상으로 제기할 수 있도록 되어 있는바, 개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위라고 할 것이므로 이 건 청구는 행정심판의 대상이 아닌 사항에 대하여 제기된 부적법한 심판청구라 할 것이다.<|sep|> <question>개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위로 보아야 하는가?<|sep|> <sys>
[30005, 3706, 10135, 712, 480, 21, 510, 480, 20, 940, 903, 480, 22, 510, 480, 20, 940, 285, 2642, 274, 554, 6690, 15, 1973, 10135, 296, 1973, 681, 270, 24639, 272, 2643, 407, 1097, 274, 2708, 1008, 7506, 2599, 285, 17574, 285, 1493, 441, 353, 1884, 307, 274, 18735, 272, 1973, 5048, 276, 1216, 339, 2013, 482, 365, 327, 954, 740, 348, 327, 272, 877, 15, 1692, 270, 1354, 1515, 363, 2580, 276, 2439, 1137, 1281, 16360, 276, 7174, 284, 272, 13386, 285, 3019, 272, 5069, 398, 2083, 274, 2708, 3019, 625, 823, 388, 270, 3121, 307, 757, 4028, 272, 1973, 10135, 285, 1216, 270, 1577, 3968, 274, 6615, 520, 2013, 551, 6981, 712, 288, 6028, 8681, 346, 823, 388, 270, 267, 17, 

'개인이 금융기관에서 자금을 대출받거나 대출금을 상환하는 일련의 행위는 사법상 계약에 관한 행위로 보아야 하므로, 이 건 청구는 행정심판의 대상이 아닌 사항에 대하여 제기된 부적법한 심판청구로 보아야 한다.'

In [ ]:
custom_pipeline(df['question'][222], df['answer'][222])

<context>토지에 대하여 도로로서의 도시계획시설결정 및 지적승인만 있었을 뿐 그 도시계획사업이 실시되었거나 그 토지가 자연공로로 이용된 적이 없는 경우에는 도시계획결정 및 지적승인의 고시만으로는 아직 공용개시행위가 있었다고 할 수 없어 그 토지가 행정재산이 되었다고 할 수 없다.<|sep|> <question>도로가 행정재산이 되기 위한 요건 및 토지에 대하여 도로로서의 도시계획시설결정 및 지적승인이 있다는 사정만으로 그 토지가 행정재산이 되는가?<|sep|> <sys>
[30005, 8959, 274, 6615, 520, 2702, 2048, 285, 1606, 2203, 1549, 6270, 903, 1450, 11825, 380, 327, 450, 276, 1708, 353, 1606, 2203, 1129, 270, 1768, 605, 450, 1281, 353, 4222, 293, 1975, 504, 286, 286, 1250, 551, 761, 270, 550, 272, 955, 274, 272, 1606, 2203, 6270, 903, 1450, 11825, 285, 8189, 380, 339, 272, 1711, 16214, 24966, 3644, 293, 327, 450, 682, 823, 365, 550, 348, 353, 4222, 293, 1973, 6201, 270, 740, 450, 682, 823, 365, 550, 267, 17, 3, 224, 30003, 2864, 293, 1973, 6201, 270, 740, 316, 992, 6877, 903, 4222, 274, 6615, 520, 2702, 2048, 285, 1606, 2203, 1549, 6270, 903, 1450, 11825, 270, 327, 590, 4100, 380, 339, 353, 4222, 293, 1973, 6201, 270, 740, 2579, 34, 3, 224, 30004]


'토지에 대하여 도로로서의 도시계획시설결정 및 지적승인이 있다는 사정만으로는 아직 공용개시행위가 있었다고 할 수 없어 그 토지가 행정재산이 되지 않는다.'

#하이브리드 검색

## 의미 검색

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
embeddings = sentence_model.encode(df['question']) #임베딩
embeddings.shape

In [ ]:
def dense_vector_search(query, k):
    emd = sentence_model.encode(query) #임베딩
    sim_scores = [cos_sim(embeddings[i], emd) for i in range(len(df))]
    index = range(0, len(df))
    pairs = zip(sim_scores, index)
    result = sorted(pairs, reverse=True)[:k]
    return zip(*result)

## 키워드 검색

In [ ]:
import math
import numpy as np
from typing import List
from transformers import PreTrainedTokenizer
from collections import defaultdict

class BM25:
  def __init__(self, corpus:List[List[str]], tokenizer:PreTrainedTokenizer):
    self.tokenizer = tokenizer
    self.corpus = corpus
    self.tokenized_corpus = self.tokenizer(corpus, add_special_tokens=False)['input_ids']
    self.n_docs = len(self.tokenized_corpus)
    self.avg_doc_lens = sum(len(lst) for lst in self.tokenized_corpus) / len(self.tokenized_corpus)
    self.idf = self._calculate_idf()
    self.term_freqs = self._calculate_term_freqs()

  def _calculate_idf(self):
    idf = defaultdict(float)
    for doc in self.tokenized_corpus:
      for token_id in set(doc):
        idf[token_id] += 1
    for token_id, doc_frequency in idf.items():
      idf[token_id] = math.log(((self.n_docs - doc_frequency + 0.5) / (doc_frequency + 0.5)) + 1)
    return idf

  def _calculate_term_freqs(self):
    term_freqs = [defaultdict(int) for _ in range(self.n_docs)]
    for i, doc in enumerate(self.tokenized_corpus):
      for token_id in doc:
        term_freqs[i][token_id] += 1
    return term_freqs

  def get_scores(self, query:str, k1:float = 1.2, b:float=0.75):
    query = self.tokenizer([query], add_special_tokens=False)['input_ids'][0]
    scores = np.zeros(self.n_docs)
    for q in query:
      idf = self.idf[q]
      for i, term_freq in enumerate(self.term_freqs):
        q_frequency = term_freq[q]
        doc_len = len(self.tokenized_corpus[i])
        score_q = idf * (q_frequency * (k1 + 1)) / ((q_frequency) + k1 * (1 - b + b * (doc_len / self.avg_doc_lens)))
        scores[i] += score_q
    return scores

  def get_top_k(self, query:str, k:int):
    scores = self.get_scores(query)
    top_k_indices = np.argsort(scores)[-k:][::-1]
    top_k_scores = scores[top_k_indices]
    return top_k_scores, top_k_indices

b_tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')
bm25 = BM25(list(df['question']), b_tokenizer)

## 하이브리드 검색

In [ ]:
from collections import defaultdict
#순위 조합 함수
def reciprocal_rank_fusion(rankings:List[List[int]], k):
    rrf = defaultdict(float)
    for ranking in rankings:
        for i, doc_id in enumerate(ranking, 1):
            rrf[doc_id] += 1.0 / (k + i)
    return sorted(rrf.items(), key=lambda x: x[1], reverse=True)

In [ ]:
def hybrid_search(query, k=20):
  d, dense_search_ranking = dense_vector_search(query, 100)#의미 검색
  b, bm25_search_ranking = bm25.get_top_k(query, 100) #bm25 검색
  print(d[:3])
  print(b[:3])
  results = reciprocal_rank_fusion([dense_search_ranking, bm25_search_ranking], k=k) #순위 조합
  return results

In [ ]:
query = '공무수행사도 청탁금지법의 대상이 되는지?'
results = hybrid_search(query)
for idx, score in results[:3]:
  print(df['question'][idx])
  print(df['answer'][idx])
  print()

[71.64245 82.17527 89.37225]
[34.04377478 33.72945782 32.53324494]
청탁금지법상 공무수행사인의 범위는 어떻게 되나요?
청탁금지법상 공무수행사인의 범위는 각종 법령에 따라 설치된 위원회의 공직자등이 아닌 위원, 권한을 위임·위탁받은 법인·단체 또는 그 기관이나 개인, 공무 수행을 위해 민간부문에서 공공기관에 파견 나온 사람, 법령에 따라 공무상 심의·평가 등을 하는 개인 또는 법인·단체 중 어느 하나에 해당하는 자가 공무수행사인으로서 법 적용대상자에 해당합니다.

공무수행사인의 경우 청탁금지법이 적용되는 범위는 어떠한가요?
공무수행사인은 공무수행에 관하여 금품등을 수수한 경우 형사처벌의 대상이 됩니다.

청탁금지법상 공무수행사인이 되는 경우 수탁된 공무 외에 다른 업무와 관련하여 서도 청탁금지법이 적용되나요?
공무 외 다른업무와 관련해서는 적용되지 않습니다.



In [ ]:
query = '상대방의 동의 없는 녹음 파일이 증거로 사용될 수 있는지'
results = hybrid_search(query)
for idx, score in results[:3]:
  print(df['question'][idx])
  print(df['answer'][idx])
  print()

[154.3777  156.18091 161.96002]
[29.35363036 27.24997709 24.07508301]
채무자와의 대화를 몰래 녹음한 테이프도 증거로 될 수 있는지
어떠한 사람이나 물건을 증거로 할 수 있는가 하는 물음에는 두 가지 의미가 포함되어 있다고 할 것입니다.하나는 그 사람이나 물건이 증거방법으로서 증거조사의 대상이 될 자격이 있는가 하는 것으로서 이를 ‘증거능력’이라 하고, 다음은 증거자료가 입증을 요하는 사실의 인정에 미치는 정도가 어떠한가 하는 것으로서 이를 ‘증명력’ 또는 ‘증거가치’라고도 합니다. 다시 말하면 전자는 증거가 될 자격이 있느냐의 문제이고, 후자는 그 증거에 의하여 당사자가 주장하는 사실을 인정할 수 있는가 하는 문제라고 말할 수 있습니다.우리 민사소송법에서는 자유심증주의를 채택하고 있기 때문에 형사소송과 달리 증거능력에 제한은 없음이 원칙입니다. 다만, 증거능력이 인정되는 증거라도 이를 믿을 것인지 여부는 자유심증주의에 의하여 전적으로 법관의 판단에 의하게 됩니다.당사자 일방이 상대방 모르게 상대방과의 대화내용을 비밀로 녹음한 경우에 관하여 판례를 보면, 우리 민사소송법이 자유심증주의를 채택하였음을 들어 상대방 모르게 비밀로 녹음한 녹음테이프를 위법으로 수집된 증거라는 이유만으로 증거능력이 없다고는 단정할 수 없다고 하였으며(대법원 1981. 4. 14. 선고 80다2314 판결), 자유심증주의를 채택하고 있는 우리 민사소송법에서 상대방의 부지중 비밀리에 상대방의 대화를 녹음하였다는 이유만으로 그 녹음테이프가 증거능력이 없다고 단정할 수 없고, 그 채증여부는 사실심법원의 재량에 속하는 것이며, 녹음테이프에 대한 증거조사는 검증의 방법에 의하여야 하고, 당사자 일방이 녹음테이프를 녹취한 녹취문을 증거로 제출하고 이에 대하여 상대방이 부지로 인부한 경우, 그 녹취문이 오히려 상대방에게 유리한 내용으로 되어 있었다면 녹음테이프 검증 없이 녹취문의 진정성립을 인정할 수 있다고 한 바 있습니다(대법원 1998. 12

In [ ]:
query = '사실을 적시하여도 명예훼손죄가 성립되는지?'
results = hybrid_search(query)
for idx, score in results[:3]:
  print(df['question'][idx])
  print(df['answer'][idx])
  print()

[68.68198  74.19534  85.412994]
[32.25632456 30.96408591 30.70341631]
사실의 적시에 대한 증거에 관계없이 명예훼손죄가 성립될 수 있는가?
명예훼손죄의 사실의 적시란 가치판단이나 평가를 내용으로 하는 의견표현에 대치되는 개념으로서 시간과 공간적으로 구체적인 과거 또는 현재의 사실관계에 관한 보고 내지 진술을 의미하는 것이며 그 표현내용이 증거에 의한 입증이 가능한 것을 말하고 그 표현 행위가 객관적으로 상대방의 인격적 가치에 대한 사회적 평가를 저하할 만했는지를 기준으로 판단하여야 한다.

전문한 사실의 적시도 명예훼손죄에 있어서의 사실의 적시에 해당하는가?
피해자와 같은 교회에 다니는 교인들일 뿐 피해자에 관한 소문을 비밀로 지켜줄 만한 특별한 신분관계는 없었던 사정을 규지할 수 있어 피고인이 그들에게 적시한 사실은 그들을 통하여 불특정 또는 다수인에게 전파될 가능성이 충분히 있었다고 보기에 넉넉하므로 원심판결에 공연성에 관한 법리오해가 있다는 논지도 받아들일 수 없다. 그러므로 상고를 기각한다.

허위사실 적시에 의한 명예훼손은 그 적시하는 사실이 허위이면 죄가 성립되는가?
구 정보통신망 이용촉진 및 정보보호 등에 관한 법률 제61조 제2항의 정보통신망을 통한 허위사실 적시에 의한 명예훼손죄, 형법 제307조 제2항의 허위사실 적시에 의한 명예훼손죄가 성립하려면 그 적시하는 사실이 허위이어야 할 뿐 아니라, 피고인이 그와 같은 사실을 적시할 때에 적시사실이 허위임을 인식하여야 한다.



In [ ]:
query = '해고를 하려면 해고 예고를 해야 한다고 하던데, 해고 예고를 하지 않은 해고의 효력은 어떻게 되나요?'
results = hybrid_search(query)
for idx, score in results[:3]:
  print(df['question'][idx])
  print(df['answer'][idx])
  print()

[102.53174 117.56753 120.36861]
[76.58989729 66.65188845 64.65421772]
해고의 예고를 하지 않았으나 정당한 이유가 있는 해고는 효력이 있나요?
개별 근로계약에서 복무규율과 임금 등 근로조건에 관한 준칙의 내용을 담고 있으면 그 명칭에 상관없이 취업규칙에 해당한다.

근로자를 해고하려고 하는데 근로자에게 미리 해고에 대한 예고를 하여야 하나요?
예, 미리 해고예고 해야 합니다.

해고예고를 하지 않은 해고는 유효한가요?
예, 유효합니다.



## RAG

In [ ]:
def rag(query):
  result = hybrid_search(query)
  if type(result) == str:
    return result
  idx = result[0][0]
  context = df['answer'][idx]
  print(df['question'][idx])
  answer = custom_pipeline(query, context)
  return answer

In [ ]:
query = '공무수행사도 청탁금지법의 대상이 되는지?'
rag(query)

[71.64245 82.17527 89.37225]
[34.04377478 33.72945782 32.53324494]
청탁금지법상 공무수행사인의 범위는 어떻게 되나요?
<context>청탁금지법상 공무수행사인의 범위는 각종 법령에 따라 설치된 위원회의 공직자등이 아닌 위원, 권한을 위임·위탁받은 법인·단체 또는 그 기관이나 개인, 공무 수행을 위해 민간부문에서 공공기관에 파견 나온 사람, 법령에 따라 공무상 심의·평가 등을 하는 개인 또는 법인·단체 중 어느 하나에 해당하는 자가 공무수행사인으로서 법 적용대상자에 해당합니다.<|sep|> <question>공무수행사도 청탁금지법의 대상이 되는지?<|sep|> <sys>
[30005, 18667, 17556, 398, 14420, 419, 3153, 333, 285, 4242, 272, 2153, 10170, 274, 1026, 1710, 551, 3179, 1760, 7732, 968, 270, 1577, 3179, 15, 4492, 276, 12952, 525, 17091, 1137, 296, 3948, 525, 1691, 2259, 353, 2286, 1019, 1692, 15, 14420, 3079, 276, 746, 2382, 3921, 363, 2023, 1515, 274, 5181, 1750, 790, 15, 10170, 274, 1026, 14420, 398, 4744, 525, 2463, 433, 276, 423, 272, 1692, 2259, 3948, 525, 1691, 469, 1839, 1196, 274, 1785, 284, 272, 18171, 14420, 419, 3153, 333, 2599, 1008, 1702, 9722, 274, 1785, 2136, 17, 3, 224, 30003, 23707, 419, 3153, 309, 9716, 17556, 285, 1216, 270, 740, 1383, 34, 3, 224, 30004]


'공무수행사도 청탁금지법의 대상이 되며, 이는 각종 법령에 따라 설치된 위원회의 공직자등이 아닌 위원, 권한을 위임·위탁받은 법인·단체 또는 그 기관이나 개인, 공무 수행을 위해 민간부문에서 공공기관에 파견 나온 사람, 법령에 따라 공무상 심의·평가 등을 하는 개인 또는 법인·단체 중 어느 하나에 해당하는 자가 공무수행사인으로서 법 적용대상자에 해당합니다.'

In [ ]:
query = '상대방의 동의 없는 녹음 파일이 증거로 사용될 수 있는지'
rag(query)

[154.3777  156.18091 161.96002]
[29.35363036 27.24997709 24.07508301]
채무자와의 대화를 몰래 녹음한 테이프도 증거로 될 수 있는지
<context>어떠한 사람이나 물건을 증거로 할 수 있는가 하는 물음에는 두 가지 의미가 포함되어 있다고 할 것입니다.<|sep|> 하나는 그 사람이나 물건이 증거방법으로서 증거조사의 대상이 될 자격이 있는가 하는 것으로서 이를 ‘증거능력’이라 하고, 다음은 증거자료가 입증을 요하는 사실의 인정에 미치는 정도가 어떠한가 하는 것으로서 이를 ‘증명력’ 또는 ‘증거가치’라고도 합니다.<|sep|>  다시 말하면 전자는 증거가 될 자격이 있느냐의 문제이고, 후자는 그 증거에 의하여 당사자가 주장하는 사실을 인정할 수 있는가 하는 문제라고 말할 수 있습니다.<|sep|> 우리 민사소송법에서는 자유심증주의를 채택하고 있기 때문에 형사소송과 달리 증거능력에 제한은 없음이 원칙입니다.<|sep|>  다만, 증거능력이 인정되는 증거라도 이를 믿을 것인지 여부는 자유심증주의에 의하여 전적으로 법관의 판단에 의하게 됩니다.<|sep|> 당사자 일방이 상대방 모르게 상대방과의 대화내용을 비밀로 녹음한 경우에 관하여 판례를 보면, 우리 민사소송법이 자유심증주의를 채택하였음을 들어 상대방 모르게 비밀로 녹음한 녹음테이프를 위법으로 수집된 증거라는 이유만으로 증거능력이 없다고는 단정할 수 없다고 하였으며(대법원 1981.<|sep|>  4.<|sep|>  14.<|sep|>  선고 80다2314 판결), 자유심증주의를 채택하고 있는 우리 민사소송법에서 상대방의 부지중 비밀리에 상대방의 대화를 녹음하였다는 이유만으로 그 녹음테이프가 증거능력이 없다고 단정할 수 없고, 그 채증여부는 사실심법원의 재량에 속하는 것이며, 녹음테이프에 대한 증거조사는 검증의 방법에 의하여야 하고, 당사자 일방이 녹음테이프를 녹취한 녹취문을 증거로 제출하고 이에 대하여 상대방이 부지로 인부한 경우, 그 녹취문이 오히려 상대방에

'상대방의 동의 없이 녹음 파일이 증거로 사용될 수 있는지 여부는 법관의 판단에 달려 있습니다.'

In [ ]:
query = '인터넷 쇼핑몰에서 가방을 구입 후 7일 이내에 청약철회를 요구하여 물품을 반품하였는데 쇼핑몰에서 물품 대금을 현금으로 환급해 주지 않고 적립금으로 대신했습니다. 현금 환급을 요청할 수 있나요?'
rag(query)

[146.20505 156.32193 166.20206]
[51.93812148 48.46309572 47.04198403]
환불위약금에 관한 약관에서 소비자가 7일 이내에 적법하게 청약철회권을 행사한 경우에도 일정 금액의 환불위약금을 공제하고 대금을 반환하도록 한 환불위약금 규정은 소비자에게 불리하게 정한 것으로서 무효인가?
<context>위 환불위약금 규정은 전자상거래법에서 정한 청약철회에 관한 규정을 위반하여 소비자에게 불리하게 정한 것으로서 무효라고 한 사례이다.<|sep|> <question>인터넷 쇼핑몰에서 가방을 구입 후 7일 이내에 청약철회를 요구하여 물품을 반품하였는데 쇼핑몰에서 물품 대금을 현금으로 환급해 주지 않고 적립금으로 대신했습니다. 현금 환급을 요청할 수 있나요?<|sep|> <sys>
[30005, 500, 13074, 500, 779, 515, 2642, 296, 2917, 398, 14762, 363, 8325, 4315, 27991, 274, 2708, 2642, 276, 2880, 13473, 2071, 666, 3247, 284, 379, 8325, 388, 2599, 9755, 625, 394, 2271, 270, 267, 17, 3, 224, 30003, 5808, 7125, 363, 6613, 276, 3523, 726, 831, 393, 4246, 274, 4315, 27991, 301, 1539, 13473, 7097, 276, 26582, 284, 759, 829, 7125, 363, 7097, 10167, 276, 4232, 339, 12047, 310, 15809, 536, 283, 20084, 339, 2155, 357, 827, 17, 4232, 12047, 276, 2357, 482, 365, 327, 4389, 34, 3, 224, 30004]


'현금 환급을 요청할 수 있습니다.'